# 1. Importing libraries

In [4]:
# !conda install -c conda-forge feature_engine -y
# !pip install xgboost==1.7.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 63.1 MB/s  0:00:03m0:00:0100:01


In [51]:
# importing function and classes needed for preprocessor
import warnings

from myclasses import *
from mymethods import *

import numpy as np
import pandas as pd
import joblib

import feature_engine

import boto3
import os

import xgboost

import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.tuner import (
    IntegerParameter,
    ContinuousParameter,
    HyperparameterTuner
)

from sklearn.metrics import r2_score

# 2. display setting 

In [6]:
pd.set_option("display.max_columns", None)

In [7]:
warnings.filterwarnings("ignore")

# 3. Reading the data

In [8]:
train = pd.read_csv("train.csv")

In [9]:
val = pd.read_csv("val.csv")

In [10]:
test = pd.read_csv("test.csv")

In [11]:
train.head()

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-05-15,Mumbai,Hyderabad,19:30:00,20:45:00,75,0.0,No Info,3100
1,Jet Airways,2019-06-27,Delhi,Cochin,13:25:00,19:00:00,1775,2.0,No Info,12819
2,Jet Airways,2019-06-03,Delhi,Cochin,21:25:00,19:00:00,1295,2.0,No Info,13292
3,Jet Airways,2019-04-01,Kolkata,Banglore,06:30:00,04:40:00,1330,1.0,In-Flight Meal Not Included,7064
4,Air India,2019-05-18,Kolkata,Banglore,16:50:00,13:45:00,1255,2.0,No Info,13432


In [14]:
test.head()

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-06,Banglore,New Delhi,08:00:00,08:15:00,1455,1,No Info,17996
1,Spicejet,2019-06-06,Kolkata,Banglore,22:20:00,00:40:00,140,0,No Info,3873
2,Indigo,2019-03-18,Kolkata,Banglore,05:30:00,08:20:00,170,0,No Info,4462
3,Indigo,2019-06-27,Chennai,Kolkata,19:35:00,21:55:00,140,0,No Info,3597
4,Indigo,2019-05-06,Kolkata,Banglore,15:15:00,17:45:00,150,0,No Info,4804


In [15]:
val.head()

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-24,Delhi,Cochin,20:25:00,01:30:00,305,1,No Info,5054
1,Multiple Carriers,2019-06-12,Delhi,Cochin,09:45:00,22:30:00,765,1,No Info,9646
2,Jet Airways,2019-03-12,Banglore,New Delhi,22:55:00,15:15:00,980,1,In-Flight Meal Not Included,11087
3,Multiple Carriers,2019-06-06,Delhi,Cochin,13:00:00,21:00:00,480,1,No Info,13587
4,Jet Airways,2019-05-18,Delhi,Cochin,23:05:00,04:25:00,1760,2,No Info,16704


# 4. Preprocessing the data

In [12]:
# loading the preprocessor
pre = joblib.load("preprocessor.pkl")

In [33]:
# fitting the preprocessor

pre.fit(
    train.drop(columns="price"),
    train.price.copy()
)

,steps,"[('ct', ...), ('selector', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('air', ...), ('doj', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [34]:
# checking if preprocessor working or not
pre.transform(train.drop(columns="price"))

,air__airline_Indigo,air__airline_Jet Airways,air__airline_other,doj__date_of_journey_week,doj__date_of_journey_day_of_year,location__source,location__destination,dur__duration_rbf_25,dur__duration_cat,dur__duration_over_1000,dur__duration,stops__total_stops,stops__is_direct_flight
0,0.0,0.0,0.0,0.647059,0.635593,-1.889834,-0.738999,-0.364119,0.0,0,-1.115266,0.0,1
1,0.0,1.0,0.0,1.000000,1.000000,1.065138,1.061367,-0.364119,2.0,1,2.290314,2.0,0
2,0.0,1.0,0.0,0.823529,0.796610,1.065138,1.061367,-0.364119,2.0,1,1.328738,2.0,0
3,0.0,1.0,0.0,0.294118,0.262712,-0.210080,-0.229582,-0.364119,2.0,1,1.398853,1.0,0
4,0.0,0.0,0.0,0.647059,0.661017,-0.210080,-0.229582,-0.364119,2.0,1,1.248607,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6271,0.0,1.0,0.0,0.176471,0.169492,1.065138,1.061367,-0.364119,2.0,1,2.590806,2.0,0
6272,0.0,0.0,0.0,0.529412,0.516949,-0.210080,-0.229582,-0.364119,2.0,0,-0.173723,1.0,0
6273,0.0,1.0,0.0,0.764706,0.779661,1.065138,1.061367,-0.364119,1.0,0,-0.664527,1.0,0
6274,0.0,0.0,1.0,1.000000,0.974576,1.065138,1.061367,-0.364119,1.0,0,-0.604429,1.0,0


# 5. Preprocess the data and load it to bucket

In [17]:
BUCKET_NAME = "sagemaker-ml-project2-flight-bucket"
DATA_PREFIX = "data"

In [45]:
def get_file_name(name):
    return f"{name}-pre.csv"

In [46]:
def export_data(data, name, pre):
    # split data into X and y subsets
    X = data.drop(columns="price")
    y = data.price.copy()
    
    # transformation
    X_pre = pre.transform(X)
    
    # exporting
    file_name = get_file_name(name)
    (
        y
        .to_frame()
        .join(X_pre)
        .to_csv(file_name, index=False, header=False)
    )

In [38]:
def upload_to_bucket(name):
    file_name = get_file_name(name)
    
    (
        boto3
        .Session()
        .resource("s3")
        .Bucket(BUCKET_NAME)
        .Object(os.path.join(DATA_PREFIX, f"{name}/{name}.csv"))
        .upload_file(file_name)
    )

In [39]:
def export_and_upload_bucket(data, name, pre):
    export_data(data, name, pre)
    upload_to_bucket(name)

In [44]:
export_and_upload_bucket(train, "train", preprocessor)

In [45]:
export_and_upload_bucket(val, "val", preprocessor)

In [46]:
export_and_upload_bucket(test, "test", preprocessor)

# 6. Model training and Hyperparameter Tunning setup

In [15]:
session = sagemaker.Session()
region_name = session.boto_region_name

In [18]:
output_path = f"s3://{BUCKET_NAME}/model/output"

In [35]:
model = Estimator(
    image_uri=sagemaker.image_uris.retrieve("xgboost", region_name, "1.2-1"),
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m4.xlarge",
    volume_size=5,
    output_path=output_path,
    use_spot_instances=True,
    max_run=600,
    max_wait=600,
    sagemaker_session=session
)

In [37]:
model.set_hyperparameters(
    objective="reg:linear",
    num_round=1000,
    eta=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    alpha=0.1
)

In [38]:
hyperparameter_ranges = {
    "eta": ContinuousParameter(0.05, 0.2),
    "alpha": ContinuousParameter(0, 1),
    "max_depth": IntegerParameter(3, 5)
}

In [39]:
tuner = HyperparameterTuner(
    estimator=model,
    objective_metric_name="validation:rmse",
    hyperparameter_ranges=hyperparameter_ranges,
    strategy="Bayesian",
    objective_type="Minimize"
)

# 7. data channel

In [24]:
def get_data_channel(name):
    bucket_path = f"s3://{BUCKET_NAME}/{DATA_PREFIX}/{name}"
    return TrainingInput(bucket_path, content_type="csv")

In [25]:
train_data_channel = get_data_channel("train")
train_data_channel

In [26]:
val_data_channel = get_data_channel("val")

In [27]:
data_channels = {
    "train": train_data_channel,
    "validation": val_data_channel
}

# 8. train and tune the model

In [40]:
tuner.fit(data_channels)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


..................................!


# 9. Model Evaluation

In [42]:
with open("xgboost-model", "rb") as f:
    best_model = joblib.load(f)
    
best_model

[10:30:19] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[10:30:19] WARNING: ../src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



In [52]:
def evaluate_model(name):
    file_name = get_file_name(name)
    data = pd.read_csv(file_name)
    
    X = xgboost.DMatrix(data.iloc[:, 1:])
    y = data.iloc[:, 0].copy()
    
    pred = best_model.predict(X)
    
    return r2_score(y, pred)

In [53]:
evaluate_model("val")

0.7131688594818115